In [ ]:
# Load Bark TTS model using HuggingFace transformers
from transformers import pipeline
import torch

try:
    # Use Bark for text-to-speech
    pipe = pipeline("text-to-speech", model="suno/bark-small", torch_dtype=torch.float16, device="cpu")
    print("Bark TTS model loaded successfully!")

    # Test with sample text
    text = "Hello, this is a test of the Bark text-to-speech model."
    speech = pipe(text)

    print(f"Generated audio with sample rate: {speech['sampling_rate']}")
    print(f"Audio shape: {speech['audio'].shape}")

except Exception as e:
    print(f"Error loading Bark model: {e}")
    print("Installing required dependencies might be needed: pip install transformers torch torchaudio")

config.json:   0%|          | 0.00/8.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/4.91k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/353 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cpu
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Bark TTS model loaded successfully!
